In [ ]:
from google.colab import files
files.upload()

In [ ]:
ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import time
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM # Use tf.keras.layers
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep

In [ ]:
df = pd.read_csv('drive/MyDrive/TimesNet/aapl.csv')
df = df.drop(["Date"], axis=1)
df.head()

In [ ]:
# 1) Clone THU’s repo
!git clone https://github.com/thuml/Time-Series-Library.git

In [ ]:
import sys
sys.path.append('/content/Time-Series-Library/')

In [ ]:
# 2) Imports
import torch, numpy as np, pandas as pd
from types import SimpleNamespace
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from models.TimesNet import Model as TimesNet      # class is named Model inside TimesNet.py :contentReference[oaicite:1]{index=1}
import matplotlib.pyplot as plt

In [ ]:
# 3) Load your CSV (e.g. aapl.csv)
#from google.colab import files
#uploaded = files.upload()         # pick aapl.csv
df_raw = pd.read_csv("drive/MyDrive/TimesNet/aapl.csv")

In [ ]:
# 4) Build time-feature matrix [month, day, weekday, hour]
#    (Daily data → hour=0)
df = df_raw.copy()
dates = pd.to_datetime(df['Date'])
time_feats = np.stack([
    dates.dt.month.values,
    dates.dt.day.values,
    dates.dt.weekday.values,
    np.zeros(len(dates), dtype=int)
], axis=1)  # shape (T,4)

In [ ]:
# 5) Drop non-numeric columns & scale
df = df.drop(columns=['Date','Scaled_sentiment'], errors='ignore')
features = df.columns.tolist()
data = df.values.astype(np.float32)
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [ ]:
# 6) Create sliding windows for X, x_mark, and y
SEQ_LEN, LABEL_LEN, PRED_LEN = 24, 24, 1
X, X_mark, y = [], [], []
close_idx = features.index('Close') if 'Close' in features else -1

for i in range(len(data) - SEQ_LEN - PRED_LEN + 1):
    X.append(data[i : i+SEQ_LEN])
    X_mark.append(time_feats[i : i+SEQ_LEN])
    y.append(data[i+SEQ_LEN : i+SEQ_LEN+PRED_LEN, close_idx])

X      = np.stack(X)       # (N, SEQ_LEN, n_feat)
X_mark = np.stack(X_mark)  # (N, SEQ_LEN, 4)
y      = np.stack(y)       # (N, PRED_LEN)

In [ ]:
# 7) Train/test split & tensorize
X_tr, X_te, M_tr, M_te, y_tr, y_te = train_test_split(
    X, X_mark, y, test_size=0.2, shuffle=False
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_tr = torch.from_numpy(X_tr).to(device)
M_tr = torch.from_numpy(M_tr).to(device)
y_tr = torch.from_numpy(y_tr).to(device)

X_te = torch.from_numpy(X_te).to(device)
M_te = torch.from_numpy(M_te).to(device)
y_te = torch.from_numpy(y_te).to(device)

In [ ]:
# 8) Build config (must include top_k & num_kernels) and model
cfg = SimpleNamespace(
    task_name='short_term_forecast',
    seq_len=SEQ_LEN, label_len=LABEL_LEN, pred_len=PRED_LEN,
    enc_in=len(features), dec_in=len(features), c_out=1,
    d_model=32, d_ff=64, embed='fixed', freq='d',  # daily freq
    dropout=0.1, e_layers=3, top_k=3, num_kernels=3
)
model = TimesNet(cfg).to(device)
print("✅ TimesNet params:", sum(p.numel() for p in model.parameters()))


In [ ]:
# 9) Train
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()

for ep in range(1, 15):
    model.train()
    perm = torch.randperm(X_tr.size(0))
    tot_loss = 0.0
    for i in range(0, len(perm), 64):
        idx = perm[i:i+64]
        xb, mb, yb = X_tr[idx], M_tr[idx], y_tr[idx]
        opt.zero_grad()
        out = model(xb, mb, None, None).squeeze(-1)   # pass dummy decoder args :contentReference[oaicite:2]{index=2}
        loss = loss_fn(out, yb)
        loss.backward()
        opt.step()
        tot_loss += loss.item() * xb.size(0)
    print(f"Epoch {ep}/30 — Train MSE: {tot_loss/len(X_tr):.6f}")

In [ ]:
# 10) Eval & plot
batch_size = 64  # կամ ավելի փոքր՝ 64, 32՝ եթե պետք լինի

model.eval()
predictions = []
true_values = []

with torch.no_grad():
    for i in range(0, len(X_te), batch_size):
        xb = X_te[i:i+batch_size]
        mb = M_te[i:i+batch_size]
        yb = y_te[i:i+batch_size]

        out = model(xb, mb, None, None).squeeze(-1)  # [batch_size, pred_len]
        predictions.append(out.cpu().numpy())
        true_values.append(yb.cpu().numpy())

# Միավորում ենք բոլոր բաչերի արդյունքները
pred = np.concatenate(predictions, axis=0)
true = np.concatenate(true_values, axis=0)


In [ ]:
# 1. Սեղմում ենք pred-ը եթե պետք է
if pred.ndim == 3 and pred.shape[1] == 1:
    pred = pred.squeeze(1)

# 2. Սեղմում ենք true-ն եթե պետք է
if true.ndim == 3 and true.shape[1] == 1:
    true = true.squeeze(1)

# 3. Եթե true միաչափ է, դարձնել 2D
if true.ndim == 1:
    true = true.reshape(-1, 1)

# 4. Պատրաստում ենք padding
pad_pred = np.zeros((pred.shape[0], len(features) - pred.shape[1]))  # (137016, 8)
pad_true = np.zeros((true.shape[0], len(features) - true.shape[1]))  # (137016, 8)

# 5. Հորիզոնական միացնում ենք
pred_full = np.hstack([pad_pred, pred])    # (137016, 9)
true_full = np.hstack([pad_true, true])     # (137016, 9)

# 6. Inverse transform
pred_close = scaler.inverse_transform(pred_full)[:, -1]
true_close = scaler.inverse_transform(true_full)[:, -1]


In [ ]:
plt.figure(figsize=(12,4))
plt.plot(true_close, label='Ground Truth')
plt.plot(pred_close, label='Prediction')
plt.title('TimesNet Close-Price Forecast')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Հաշվարկում ենք մետրիկաները
mse = mean_squared_error(true_close, pred_close)
rmse = np.sqrt(mse)
mae = mean_absolute_error(true_close, pred_close)
r2 = r2_score(true_close, pred_close)

# Տպում ենք արդյունքները
print(f"✅ Test MSE:  {mse:.6f}")
print(f"✅ Test RMSE: {rmse:.6f}")
print(f"✅ Test MAE:  {mae:.6f}")
print(f"✅ Test R²:   {r2:.6f}")
